# Propuesta de sistema de clasificación

Obtenemos el error de las polilíneas (que nos han proporcionado) como la distancia cuadrática de cada polilínea a los datos.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# 3D Libs
import open3d as o3d
import laspy
print(laspy.__version__)

# Geospatial libs
import rasterio
import alphashape as ash
import geopandas as gpd
import shapely as sh    

from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.features import shapes
from shapely.geometry import Polygon

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import SpectralClustering

2.5.3


In [12]:
def get_coord(points):
    
    x_vals = [punto[0] for punto in points]
    y_vals = [punto[1] for punto in points]
    z_vals = [punto[2] for punto in points]
    
    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

def get_coord2(extremos_apoyos):
    
    x_vals = [extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"]]
    y_vals = [extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"]]
    z_vals = extremos_apoyos[0]["COORDENADAS_Z"] + extremos_apoyos[1]["COORDENADAS_Z"]

    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

def unravel_data_element(element):
    
    for key in element.keys():
    
        if type(element[key]) in [list, dict]:
            
            print(f"\n{key}: ")
            
            if type(element[key]) == list:
            
                element2 = element[key][0]
                print(f"- Length of list: {len(element[key])}")
                
            else:
                
                element2 = element[key]
            
            for key2 in element2.keys():
                
                print(f"    {key2}: {element2[key2]}")
                
                if type(element2[key2]) == list:
                    print(f"    - Length of list: {len(element2[key2])}")
        
        else:
            print(f"\n{key}: {element[key]}")

Obtenemos los datos

In [13]:
import json

path = "../data/"

with open(path+'vanos.json', 'r') as archivo:
    data1 = json.load(archivo)

with open(path+'datos1.json', 'r') as archivo:
    data2 = json.load(archivo)

with open(path+'datos2.json', 'r') as archivo:
    data3 = json.load(archivo)

Función de rotación

In [14]:
def rotate_points(points, extremos_values):
    
    points = np.array(points).T

    extremo1 = np.array(extremos_values).T[0]  # Extremo superior del primer poste
    extremo2 = np.array(extremos_values).T[2]  # Extremo inferior del primer poste
    
    # Calcular la distancia en el plano XY y la dirección de la diagonal
    distancia_xy = np.linalg.norm(extremo2[:2] - extremo1[:2])
    direccion_diagonal = (extremo2[:2] - extremo1[:2]) / distancia_xy # Normalizada para la distancia
    
    # Calcular el ángulo de rotación necesario para alinear la diagonal con el eje Y
    angulo = np.arctan2(direccion_diagonal[1], direccion_diagonal[0])
    
    # Ajustar el ángulo para la rotación correcta
    angulo += np.pi / 2
    c, s = np.cos(angulo), np.sin(angulo)
    
    # Crear la matriz de rotación para alinear la diagonal con el eje Y
    matriz_rotacion = np.array([[c, s, 0],
                                [-s, c, 0],
                                [0, 0, 1]])
    
    rotated_points = matriz_rotacion.dot(points.T)
    # print(rotated.shape)
    
    return matriz_rotacion, np.array(rotated_points)

Error para la clasificación

In [15]:
def rmse(x, y):
    return np.sqrt(np.mean((x-y)**2))

Función que obtiene el error de cada polilínea

In [16]:
def error(data, vano):

    puntos_conductores = data[vano]['LIDAR']['CONDUCTORES']
    puntos_vertices = data[vano]['CONDUCTORES'][0]['VERTICES']
    puntos_vertices2 = data[vano]['CONDUCTORES'][1]['VERTICES']
    puntos_vertices3 = data[vano]['CONDUCTORES'][2]['VERTICES']
    puntos_extremos = data[vano]['APOYOS']

    x_vals_conductores, y_vals_conductores, z_vals_conductores = get_coord(puntos_conductores)
    x_vals_extremos, y_vals_extremos, z_vals_extremos = get_coord2(puntos_extremos)
    x_vert1, y_vert1, z_vert1 = get_coord(puntos_vertices)
    x_vert2, y_vert2, z_vert2 = get_coord(puntos_vertices2)
    x_vert3, y_vert3, z_vert3 = get_coord(puntos_vertices3)

    cond_values = [x_vals_conductores, y_vals_conductores, z_vals_conductores]
    extremos_values = [x_vals_extremos, y_vals_extremos, z_vals_extremos]
    vert_values1 = [x_vert1, y_vert1, z_vert1]
    vert_values2 = [x_vert2, y_vert2, z_vert2]
    vert_values3 = [x_vert3, y_vert3, z_vert3]

    mat, rotated_conds = rotate_points(cond_values, extremos_values)
    extremos_values = mat.dot(extremos_values)
    rotated_vertices1 = mat.dot(vert_values1)
    rotated_vertices2 = mat.dot(vert_values2)
    rotated_vertices3 = mat.dot(vert_values3)

    X_extremos = extremos_values[0]
    Y_extremos = extremos_values[1]
    Z_extremos = extremos_values[2]

    X_cond = rotated_conds[0]
    Y_cond = rotated_conds[1]
    Z_cond = rotated_conds[2]

    x = []
    y = []
    z = []

    for i in range(len(X_cond)):
        if Y_cond[i] > np.min(Y_extremos) and Y_cond[i] < np.max(Y_extremos):
            x.append(X_cond[i])
            y.append(Y_cond[i])
            z.append(Z_cond[i])

    x_cond = np.array(x)
    y_cond = np.array(y)
    z_cond = np.array(z)

    [X, y] = [x_cond.reshape(-1, 1), y_cond.reshape(-1, 1)]

    model = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=0)

    y_spectral = model.fit_predict(X)

    x1, x2, x3 = [], [], []
    y1, y2, y3 = [], [], []
    z1, z2, z3 = [], [], []

    for i in range(0, len(y_spectral)):
        if y_spectral[i] == 0:
            x1.append(X[i])
            y1.append(y[i])
            z1.append(z[i])
        if y_spectral[i] == 1:
            x2.append(X[i])
            y2.append(y[i])
            z2.append(z[i])
        if y_spectral[i] == 2:
            x3.append(X[i])
            y3.append(y[i])
            z3.append(z[i])

    x1, x2, x3 = np.array(x1), np.array(x2), np.array(x3)
    y1, y2, y3 = np.array(y1), np.array(y2), np.array(y3)
    z1, z2, z3 = np.array(z1), np.array(z2), np.array(z3)

    print(f"Errores de las polilíneas del vano: {vano}")
    # print(np.sqrt(rmse(x1, rotated_vertices1[0])**2+rmse(y1, rotated_vertices1[1])**2+rmse(z1, rotated_vertices1[2])**2))
    # print(np.sqrt(rmse(x2, rotated_vertices2[0])**2+rmse(y2, rotated_vertices2[1])**2+rmse(z2, rotated_vertices2[2])**2))
    # print(np.sqrt(rmse(x3, rotated_vertices3[0])**2+rmse(y3, rotated_vertices3[1])**2+rmse(z3, rotated_vertices3[2])**2))
    print(rmse(x1, rotated_vertices1[0]), rmse(y1, rotated_vertices1[1]))
    print(rmse(x2, rotated_vertices2[0]), rmse(y2, rotated_vertices2[1]))
    print(rmse(x3, rotated_vertices3[0]), rmse(y3, rotated_vertices3[1]))

Función para observar las polilíneas

In [17]:
def grafica(data, vano):
    puntos_conductores = data[vano]['LIDAR']['CONDUCTORES']
    puntos_vertices = data[vano]['CONDUCTORES'][0]['VERTICES']
    puntos_vertices2 = data[vano]['CONDUCTORES'][1]['VERTICES']
    puntos_vertices3 = data[vano]['CONDUCTORES'][2]['VERTICES']
    puntos_extremos = data[vano]['APOYOS']

    x_vals_conductores, y_vals_conductores, z_vals_conductores = get_coord(puntos_conductores)
    x_vals_extremos, y_vals_extremos, z_vals_extremos = get_coord2(puntos_extremos)
    x_vert1, y_vert1, z_vert1 = get_coord(puntos_vertices)
    x_vert2, y_vert2, z_vert2 = get_coord(puntos_vertices2)
    x_vert3, y_vert3, z_vert3 = get_coord(puntos_vertices3)

    cond_values = [x_vals_conductores, y_vals_conductores, z_vals_conductores]
    extremos_values = [x_vals_extremos, y_vals_extremos, z_vals_extremos]
    vert_values1 = [x_vert1, y_vert1, z_vert1]
    vert_values2 = [x_vert2, y_vert2, z_vert2]
    vert_values3 = [x_vert3, y_vert3, z_vert3]

    # Rotamos los puntos pare ver el perfil del vano
    mat, rotated_conds = rotate_points(cond_values, extremos_values)
    extremos_values = mat.dot(extremos_values)
    rotated_vertices1 = mat.dot(vert_values1)
    rotated_vertices2 = mat.dot(vert_values2)
    rotated_vertices3 = mat.dot(vert_values3)

    X_extremos = extremos_values[0]
    Y_extremos = extremos_values[1]
    Z_extremos = extremos_values[2]

    X_cond = rotated_conds[0]
    Y_cond = rotated_conds[1]
    Z_cond = rotated_conds[2]

    # Filtramos los puntos que estén entre los extremos
    x = []
    y = []
    z = []

    for i in range(len(X_cond)):
        if Y_cond[i] > np.min(Y_extremos) and Y_cond[i] < np.max(Y_extremos):
            x.append(X_cond[i])
            y.append(Y_cond[i])
            z.append(Z_cond[i])

    x = np.array(x)
    y = np.array(y)
    z = np.array(z)

    data_2d_cond = np.column_stack((y, z))

    pca = PCA(n_components=2)
    data_2d_pca_cond = pca.fit_transform(data_2d_cond)

    y_min_cond, y_max_cond = data_2d_pca_cond[:, 1].min(), data_2d_pca_cond[:,1].max()

    f_ind = (data_2d_pca_cond[:,1] > y_min_cond) & (data_2d_pca_cond[:,1] < y_max_cond)
    x_filt_cond, y_filt_cond, z_filt_cond = x[f_ind], y[f_ind], z[f_ind]

    # Función de la catenaria
    from sklearn.preprocessing import StandardScaler
    from scipy.optimize import curve_fit
    def catenaria(x, a, h, k):
        return a*np.cosh((x-h)/a)+k

    y_vals = y_filt_cond.reshape(-1, 1)
    z_vals = z_filt_cond.reshape(-1, 1)

    scaler_y = StandardScaler()
    scaler_z = StandardScaler()

    y_vals_scaled = scaler_y.fit_transform(y_vals).flatten()
    z_vals_scaled = scaler_z.fit_transform(z_vals).flatten()

    p0 = [1, 0, 0]

    parametros, _ = curve_fit(catenaria, y_vals_scaled.flatten(), z_vals_scaled)

    # Ajuste de los puntos de los datos a una catenaria
    fitted_z_vals_scaled = catenaria(y_vals_scaled.flatten(), *parametros)
    fitted_z_vals = scaler_z.inverse_transform(fitted_z_vals_scaled.reshape(-1, 1)).flatten()

    plt.figure(figsize=(10, 6))
    # plt.scatter(y, z, label='Datos', s=30)
    plt.scatter(x, y, label='Datos', s=30)
    # plt.scatter(scaler_y.inverse_transform(y_vals_scaled.reshape(-1, 1)).flatten(), fitted_z_vals, color='red', label='Ajuste', s=5)

    # Polilínea que nos proporcionan con los datos
    plt.plot(rotated_vertices1[0], rotated_vertices1[1], "-o", color='green', label='Polilínea_datos')
    plt.plot(rotated_vertices2[0], rotated_vertices2[1], "-o", color='green')
    plt.plot(rotated_vertices3[0], rotated_vertices3[1], "-o", color='green')

    # Interpolación de la polilínea
    minimo = np.min(scaler_y.inverse_transform(y_vals_scaled.reshape(-1, 1)).flatten())
    maximo = np.max(scaler_y.inverse_transform(y_vals_scaled.reshape(-1, 1)).flatten())
    x_pol = np.linspace(minimo, maximo, 1000)

    scaler_x = StandardScaler()

    x_scaled = scaler_x.fit_transform(x_pol.reshape(-1, 1)).flatten()

    fitted_y_scaled = catenaria(x_scaled.flatten(), *parametros)
    fitted_y = scaler_z.inverse_transform(fitted_y_scaled.reshape(-1, 1)).flatten()

    y2 = np.interp(x_pol, scaler_y.inverse_transform(y_vals_scaled.reshape(-1, 1)).flatten(), fitted_z_vals, period=len(fitted_z_vals))

    # plt.plot(x_pol, y2, color='black', label='Polilínea')

    plt.legend()
    plt.title(vano)

Obtenemos los errores de cada vano

In [18]:
for i in range(0, len(data1)):
    error(data1, i)

Errores de las polilíneas del vano: 0
0.224662566450249 17.500092189135145
0.45444912693623674 18.112205987673565
0.19911906256594042 14.063167244776805


/home/paula-ubuntu/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Errores de las polilíneas del vano: 1
0.9780835191874704 32.912683867118155
2.404719965478737 33.729889285526376
0.6556405978005022 33.4841721062586


/home/paula-ubuntu/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Errores de las polilíneas del vano: 2
0.37257501319414593 21.98719223455393
0.8589787939727077 23.62180367356227
0.9707508037667418 22.993208177125414
Errores de las polilíneas del vano: 3
0.3251550042446137 38.53677192995631
0.03726756429672582 36.92391426863588
0.3450073509097988 40.3248778465898


...

Errores de las polilíneas del vano: 18
1.2706493522794096 32.4411336108123
1.270885261824709 31.854643334497112
2.639589061940925 31.14686355990721
Errores de las polilíneas del vano: 19
2.554327927734459 28.901822554214565
0.8661630527664299 25.51065189361937
0.7599336123343691 24.39115877174566
Errores de las polilíneas del vano: 20
1.8947638322913212 21.08102818538678
1.5023054822775148 26.544786470027045
0.22065944554659098 24.690009689083105


/home/paula-ubuntu/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Errores de las polilíneas del vano: 21
1.762043427664351 39.37400158554131
1.7257357257481911 38.10063540406504
0.28261776069809313 35.416198494159936


/home/paula-ubuntu/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Errores de las polilíneas del vano: 22
0.8828756497717949 30.769333596691386
0.66698430501917 44.02317187171442
0.8310950105441638 40.96879784236647
Errores de las polilíneas del vano: 23
0.8590789843404497 34.52622632663209
0.8625913427894176 31.753268891759443
0.27157514916916137 34.01325517152615
Errores de las polilíneas del vano: 24
1.6416843375401955 17.61378157562157
1.815458882730973 7.774709938123033
0.4787542595728718 10.949591415139572
